# Chapter 2: Nonlinear Oscillations and the Smale Horseshoe Map
*Based on P. J. Holmes*

## Summary

The Smale horseshoe is a canonical example of chaotic dynamics in two dimensions. Key concepts:

- **Stretching and folding**: The geometric mechanism behind chaos
- **Invariant Cantor set**: The set of points that never escape
- **Symbolic dynamics**: Bi-infinite sequences encoding all orbits
- **Homoclinic tangles**: How horseshoes arise in physical systems
- **Duffing oscillator**: A forced nonlinear oscillator exhibiting horseshoe dynamics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from ipywidgets import interact, IntSlider, FloatSlider
%matplotlib inline
plt.style.use('dark_background')

## 2.1 The Horseshoe Map

The Smale horseshoe stretches a square, bends it into a horseshoe shape, and places it back over the original square.

In [2]:
def draw_horseshoe_iteration(n_iter):
    """Visualize the horseshoe map's action on a square"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Original square
    ax = axes[0]
    square = plt.Rectangle((0, 0), 1, 1, fill=False, edgecolor='white', lw=2)
    ax.add_patch(square)
    
    # Color the vertical strips that survive iteration
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, 2**n_iter))
    width = 1 / (3**n_iter)
    
    def get_strip_positions(n):
        if n == 0:
            return [0]
        prev = get_strip_positions(n-1)
        new_pos = []
        for p in prev:
            new_pos.append(p / 3)
            new_pos.append(p / 3 + 2/3)
        return sorted(new_pos)
    
    if n_iter > 0:
        positions = get_strip_positions(n_iter)
        for i, pos in enumerate(positions[:len(colors)]):
            strip = plt.Rectangle((pos, 0), width, 1, 
                                   facecolor=colors[i % len(colors)], alpha=0.7)
            ax.add_patch(strip)
    
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.set_aspect('equal')
    ax.set_title(f'Vertical strips after {n_iter} iterations')
    
    # Horseshoe shape
    ax = axes[1]
    
    # Draw the horseshoe outline
    theta = np.linspace(0, np.pi, 50)
    r_outer, r_inner = 0.5, 0.25
    
    # Outer arc
    x_outer = 0.5 + r_outer * np.cos(theta)
    y_outer = 1 + r_outer * np.sin(theta)
    
    # Inner arc
    x_inner = 0.5 + r_inner * np.cos(theta[::-1])
    y_inner = 1 + r_inner * np.sin(theta[::-1])
    
    # Left leg
    ax.plot([0, 0], [0, 1], 'cyan', lw=2)
    ax.plot([0.25, 0.25], [0, 1], 'cyan', lw=2)
    
    # Right leg  
    ax.plot([0.75, 0.75], [0, 1], 'cyan', lw=2)
    ax.plot([1, 1], [0, 1], 'cyan', lw=2)
    
    # Top arcs
    ax.plot(x_outer, y_outer, 'cyan', lw=2)
    ax.plot(x_inner, y_inner, 'cyan', lw=2)
    
    # Original square outline
    square = plt.Rectangle((0, 0), 1, 1, fill=False, edgecolor='white', 
                           lw=2, linestyle='--', alpha=0.5)
    ax.add_patch(square)
    
    ax.set_xlim(-0.2, 1.2)
    ax.set_ylim(-0.2, 1.8)
    ax.set_aspect('equal')
    ax.set_title('Horseshoe mapping')
    
    plt.tight_layout()
    plt.show()

interact(draw_horseshoe_iteration,
         n_iter=IntSlider(min=0, max=6, step=1, value=2, description='Iterations'));

interactive(children=(IntSlider(value=2, description='Iterations', max=6), Output()), _dom_classes=('widget-in…

## 2.2 The Hénon Map

The Hénon map is a simplified model that exhibits horseshoe-like dynamics:
$$x_{n+1} = 1 - ax_n^2 + y_n$$
$$y_{n+1} = bx_n$$

In [3]:
def henon_attractor(a, b, n_points=50000):
    """Generate the Hénon attractor"""
    x, y = 0.1, 0.1
    xs, ys = [], []
    
    # Skip transient
    for _ in range(1000):
        x, y = 1 - a*x**2 + y, b*x
    
    # Collect points
    for _ in range(n_points):
        x, y = 1 - a*x**2 + y, b*x
        xs.append(x)
        ys.append(y)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(xs, ys, s=0.1, c='cyan', alpha=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'Hénon Attractor: a={a}, b={b}')
    ax.set_aspect('equal')
    plt.show()

interact(henon_attractor,
         a=FloatSlider(min=1.0, max=1.5, step=0.01, value=1.4, description='a'),
         b=FloatSlider(min=0.1, max=0.5, step=0.01, value=0.3, description='b'));

interactive(children=(FloatSlider(value=1.4, description='a', max=1.5, min=1.0, step=0.01), FloatSlider(value=…

## 2.3 Duffing Oscillator

The forced Duffing equation $\ddot{x} + \delta\dot{x} + \alpha x + \beta x^3 = \gamma\cos(\omega t)$ exhibits chaotic behavior.

In [6]:
from scipy.integrate import odeint

def duffing_poincare(gamma=0.3, delta=0.2):
    """Poincaré section of the Duffing oscillator"""
    alpha, beta, omega = -1, 1, 1
    
    def duffing(y, t):
        x, v = y
        return [v, -delta*v - alpha*x - beta*x**3 + gamma*np.cos(omega*t)]
    
    # Time array (sample at period of forcing)
    T = 2 * np.pi / omega
    n_periods = 2000
    t = np.linspace(0, n_periods * T, n_periods * 100)
    
    # Integrate
    y0 = [0.1, 0.0]
    sol = odeint(duffing, y0, t)
    
    # Poincaré section: sample once per period
    poincare_indices = np.arange(500, n_periods) * 100
    poincare_indices = poincare_indices[poincare_indices < len(sol)]
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(sol[poincare_indices, 0], sol[poincare_indices, 1], 
               s=1, c='cyan', alpha=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('v')
    ax.set_title(f'Duffing Oscillator Poincaré Section: γ={gamma}, δ={delta}')
    plt.show()

interact(duffing_poincare,
         gamma=FloatSlider(min=0.1, max=0.5, step=0.01, value=0.3, description='γ'),
         delta=FloatSlider(min=0.1, max=0.4, step=0.01, value=0.2, description='δ'));

interactive(children=(FloatSlider(value=0.3, description='γ', max=0.5, min=0.1, step=0.01), FloatSlider(value=…

## Notes

- **Cantor set structure**: The invariant set of the horseshoe is homeomorphic to a Cantor set
- **Topological conjugacy**: The dynamics on the invariant set are equivalent to the shift map on bi-infinite binary sequences
- **Structural stability**: Small perturbations preserve the qualitative dynamics